In [ ]:
suppressMessages(library('CellChat'))

In [ ]:
suppressMessages(library('tidyverse'))
suppressMessages(library('patchwork'))
suppressMessages(library('cowplot'))

suppressMessages(library('rhdf5'))
suppressMessages(library('Seurat'))

In [ ]:
suppressMessages(library(NMF))

In [4]:
library(ggalluvial)

In [5]:
suppressMessages(library(ComplexHeatmap))

In [6]:
library("MetBrewer")

In [ ]:
suppressMessages(library(ggpubr))

In [ ]:
suppressMessages(library(future.apply))

In [9]:
options(future.rng.onMisue = "ignore", future.seed = NULL)

In [ ]:
sessionInfo()

In [11]:
options(repr.plot.width=14, 
        repr.plot.height=10,
        repr.plot.res = 200)

In [12]:
source('function_single_cell_conversion.R')

In [13]:
# cannot find the R/analysis.R file in the bin, so modified here
netClustering  <- function (object, slot.name = "netP", type = c("functional", 
    "structural"), comparison = NULL, k = NULL, methods = "kmeans", 
    do.plot = TRUE, fig.id = NULL, do.parallel = TRUE, nCores = 4, 
    k.eigen = NULL) 
{
    type <- match.arg(type)
    if (object@options$mode == "single") {
        comparison <- "single"
        cat("Classification learning of the signaling networks for a single dataset", 
            "\n")
    }
    else if (object@options$mode == "merged") {
        if (is.null(comparison)) {
            comparison <- 1:length(unique(object@meta$datasets))
        }
        cat("Classification learning of the signaling networks for datasets", 
            as.character(comparison), "\n")
    }
    comparison.name <- paste(comparison, collapse = "-")
    Y <- methods::slot(object, slot.name)$similarity[[type]]$dr[[comparison.name]]
    data.use <- Y
    if (methods == "kmeans") {
        if (!is.null(k)) {
            clusters = kmeans(data.use, k, nstart = 10)$cluster
        }
        else {
            N <- nrow(data.use)
            kRange <- seq(2, min(N - 1, 10), by = 1)
            if (do.parallel) {
                #future::plan("multiprocess", workers = nCores)
                future::plan("multisession", workers = 8)
                options(future.globals.maxSize = 1000 * 1024^2)
            }
            my.sapply <- ifelse(test = future::nbrOfWorkers() == 
                1, yes = pbapply::pbsapply, no = future.apply::future_sapply)
            results = my.sapply(X = 1:length(kRange), FUN = function(x) {
                idents <- kmeans(data.use, kRange[x], nstart = 10)$cluster
                clusIndex <- idents
                adjMat0 <- Matrix::Matrix(as.numeric(outer(clusIndex, 
                  clusIndex, FUN = "==")), nrow = N, ncol = N)
                return(list(adjMat = adjMat0, ncluster = length(unique(idents))))
            }, simplify = FALSE)
            adjMat <- lapply(results, "[[", 1)
            CM <- Reduce("+", adjMat)/length(kRange)
            res <- computeEigengap(as.matrix(CM))
            numCluster <- res$upper_bound
            clusters = kmeans(data.use, numCluster, nstart = 10)$cluster
            if (do.plot) {
                gg <- res$gg.obj
                ggsave(filename = paste0("estimationNumCluster_", 
                  fig.id, "_", type, "_dataset_", comparison.name, 
                  ".pdf"), plot = gg, width = 3.5, height = 3, 
                  units = "in", dpi = 300)
            }
        }
    }
    else if (methods == "spectral") {
        A <- as.matrix(data.use)
        D <- apply(A, 1, sum)
        L <- diag(D) - A
        L <- diag(D^-0.5) %*% L %*% diag(D^-0.5)
        evL <- eigen(L, symmetric = TRUE)
        plot(rev(evL$values)[1:30])
        Z <- evL$vectors[, (ncol(evL$vectors) - k.eigen + 1):ncol(evL$vectors)]
        clusters = kmeans(Z, k, nstart = 20)$cluster
    }
    if (!is.list(methods::slot(object, slot.name)$similarity[[type]]$group)) {
        methods::slot(object, slot.name)$similarity[[type]]$group <- NULL
    }
    methods::slot(object, slot.name)$similarity[[type]]$group[[comparison.name]] <- clusters
    return(object)
}

In [14]:
# this is where the bug is, fixing not correct matrix sparcing and fixing the weird indents everywhere.......
# https://github.com/sqjin/CellChat/issues/210#issuecomment-1538673400

subset_CellChat_single_dataset_rev = function(object, cells.use = NULL, idents.use = NULL, 
                               group.by = NULL, thresh = 0.05){
    
    labels <- object@idents # factor without names
    names(labels) = colnames(object@data) # new line to name by cell id and fix the level.use is NULL issue
    level.use0 <- levels(labels)
    
    if (!is.null(cells.use)) {
        level.use <- levels(labels)[levels(labels) %in% unique(as.character(labels[cells.use]))] 
        cells.use.index <- which(names(labels) %in% cells.use)
    }
    
    if(!is.null(idents.use)) {
        level.use <- levels(labels)[levels(labels) %in% unique(as.character(idents.use))] 
        cells.use.index <- which(labels %in% level.use)
    }
        
    if (nrow(object@data) > 0) {
        data.subset <- object@data[, cells.use.index]
    } else {
        data.subset <- matrix(0, nrow = 0, ncol = 0)
    }
    if (nrow(object@data.project) > 0) {
        data.project.subset <- object@data.project[, cells.use.index]
    } else {
        data.project.subset <- matrix(0, nrow = 0, ncol = 0)
    }
    
    data.signaling.subset <- object@data.signaling[, cells.use.index]
        
    meta.subset <- object@meta[cells.use.index, , drop = FALSE]

    cat("Update slots object@images, object@net, object@netP in a single dataset...", '\n')

    group.existing <- level.use0[level.use0 %in% level.use]
    group.existing.index <- which(level.use0 %in% level.use)

    images <- object@images
    for (images.j in names(images)) {
        values <- images[[images.j]]

        if (images.j %in% c("coordinates")) {
            values.new <- values[cells.use.index, ]
            images[[images.j]] <- values.new
        }
        if (images.j %in% c("distance")) {
            values.new <- values[group.existing.index, group.existing.index, drop = FALSE]
            images[[images.j]] <- values.new
        }
    }
    images.subset <- images


    net <- object@net
    for (net.j in names(net)) {
      values <- net[[net.j]]
      if (net.j %in% c("prob","pval")) {
        values.new <- values[group.existing.index, group.existing.index, ,drop = FALSE] # this is the bug line
        net[[net.j]] <- values.new
      }
      if (net.j %in% c("count","sum","weight")) {
        values.new <- values[group.existing.index, group.existing.index, drop = FALSE]
        net[[net.j]] <- values.new
      }
    }
    net.subset <- net
    
    # summarise related ligands and receptors
    netP = computeCommunProbPathway(net = net.subset, pairLR.use = object@LR$LRsig, thresh = thresh)
    netP$centr = netAnalysis_computeCentrality(net = netP$prob) # netP not net.subset
    netP.subset <- netP
    idents.subset <- object@idents[cells.use.index]
    idents.subset <- factor(idents.subset, levels = level.use)
    
    
    object.subset <- methods::new(
    Class = "CellChat",
    data = data.subset,
    data.signaling = data.signaling.subset,
    data.project = data.project.subset,
    images = images.subset,
    net = net.subset,
    netP = netP.subset,
    meta = meta.subset,
    idents = idents.subset,
    var.features = object@var.features,
    LR = object@LR,
    DB = object@DB,
    options = object@options
    )
    return(object.subset)
    }

In [15]:
plot_theme_tmod =  theme(title = element_text(size = 14),
                   axis.title = element_text(size = 14),
                   axis.text = element_text(size = 12),
                   legend.title =  element_text(size = 12),
                   text = element_text(size = 10),
                   plot.margin = margin(0,0,0,0,"cm")) 

In [16]:
'%notin%' = Negate('%in%')

In [17]:
data_folder = '/fast/users/twei_m/work/crc/datasets'

In [18]:
new_data_folder = '/fast/users/twei_m/work/crc/datasets_new_preprocessing'

In [19]:
imm_str_cell_order = c('B_cells','Plasma_cells','CD4+_T_cells','CD8+_T_cells','Treg_cells',
                                         'Dendritic_cells','Macrophages','Mast_cells','Monocytes','Neutrophils',
                                         'Endothelial_cells','Fibroblasts','Pericytes')

In [20]:
cell_type_order_with_Tumour_without_Monocytes = c('B_cells','Plasma_cells',
                                'CD4+_T_cells','CD8+_T_cells','Treg_cells',
                                'Dendritic_cells','Macrophages','Mast_cells',#'Monocytes',
                                                  'Neutrophils',
                                'Endothelial_cells','Fibroblasts','Pericytes',
                                'Tumour', 'Stem','Stem_TA','Tuft',
                                'Immature_Goblet','Goblet','Enterocyte_progenitor','Enterocytes'

                                )

In [21]:
cell_type_order_with_Tumour_without_Monocytes_color = c('#7d8df0', '#AA82A7',
                                                        '#335B8E', '#79c9c9', '#c44545',
                                                        '#67C9FF', '#cf7708', '#B5B867', #  '#DEB058'
                                                        '#899DA4',
                                                         '#CE9486', '#8f7022','#1aa375',
                                                        '#AA0000', '#00CCFF', '#0072b1','#FF9900',
                                                        '#990099', '#660099','#999900', '#336600'
                                                        
)

In [22]:
Numbat_and_Scitcem = read.csv('../datasets_new_preprocessing/20231108_Numbat_and_Scitcem_resolved_assignment_cellid.csv',
                             row.names=1)

In [23]:
str_imm_id_tumour = read.csv('../datasets_new_preprocessing/str_and_imm_cellid_tumour.csv',
                             row.names=1)

In [24]:
str_imm_id_normal = read.csv('../datasets_new_preprocessing/str_and_imm_cellid_normal.csv',
                             row.names=1)

### Read h5 and transform into Seurat

In [25]:
h5_path = '../datasets_new_preprocessing/202307_adata_all_full_cell_type_annotation.h5'

In [26]:
adata_h5 = H5Fopen(h5_path)

In [ ]:
# main matrix
adata_all = read_scanpy_h5(h5_path,
                           if_pca = TRUE, if_umap = TRUE, if_CB_counts=TRUE, if_raw_counts=FALSE)

In [ ]:
adata_all

In [29]:
# get sample label
cat = c(adata_h5$obs$'sample'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'sample'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'sample')

In [30]:
# get patient 
cat = c(adata_h5$obs$'patient'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'patient'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'patient')

In [31]:
# get sample_origin 
cat = c(adata_h5$obs$'sample_origin'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'sample_origin'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'sample_origin')

In [32]:
# get MS_status
cat = c(adata_h5$obs$'MS_status'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'MS_status'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'MS_status')

In [33]:
# get cell type 1a 
cat = c(adata_h5$obs$'celltype_1a'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'celltype_1a'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'celltype_1a')

In [34]:
# get cell type 2 
cat = c(adata_h5$obs$'cell_type_level2'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'cell_type_level2'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'cell_type_level2')


In [35]:
h5closeAll()

In [ ]:
# cell type in Uhlitz 
cell_type_list = c(unique(adata_all@meta.data['cell_type_level2'])$cell_type_level2) %>% sort()
cell_type_list = gsub('/', '_', cell_type_list)
cell_type_list = gsub(' ', '_', cell_type_list)
cell_type_list

In [37]:
# replace the / cuz it will cuz a file saving problem
adata_all@meta.data  = adata_all@meta.data %>% mutate(cell_type_level2 = gsub('/', '_', cell_type_level2)) %>%
mutate(cell_type_level2 = gsub(' ', '_', cell_type_level2))

#### cell type annotation

In [41]:
adata_all@meta.data = adata_all@meta.data %>% 
merge(Numbat_and_Scitcem, by = 0, all.x = TRUE)  %>% # this will do rownames to column
mutate(cell_type_cell_identity = case_when(is.na(cell_identity) ~ cell_type_level2,
                                           cell_identity == 'normal_sample' ~ cell_type_level2,
                                           .default = paste0(cell_type_level2, '_', cell_identity))) %>%
column_to_rownames('Row.names')

### Create a cell type / group label that can later separate NT and TT

In [42]:
# all genomically tumour as tumour
adata_all@meta.data = adata_all@meta.data %>% 
mutate(cell_identity_resolved_tumour = case_when(is.na(cell_identity) ~ cell_type_level2,
                                                 cell_identity == 'genomically_tumour' ~ 'Tumour',
                                                 .default = cell_type_level2))
                                                 #cell_identity == 'normal_sample' ~ cell_type_level2,

                                                 #.default = paste0(cell_type_level2, '_', cell_identity)))


In [44]:
# subset tumour samples vs normal sample
adata_GT_TME = subset(adata_all, subset = sample_origin == 'tumour' & 
                      cell_identity %in% c(NA, 'genomically_tumour') &
                      cell_type_level2 != 'Monocytes')

adata_GN_noTCs = subset(adata_all, subset = sample_origin == 'tumour' & cell_identity == 'genomically_normal' &
                 cell_type_level2 %notin% c('TC1', 'TC2', 'TC3', 'TC4', 'Monocytes'))

adata_n_noTCs = subset(adata_all, subset = sample_origin == 'normal' & 
                       cell_type_level2 %notin% c('TC1', 'TC2', 'TC3', 'TC4', 'Monocytes'))

In [45]:
adata_all@meta.data = adata_all@meta.data %>% mutate(cell_id = rownames(adata_all@meta.data))

In [46]:
most_DC2_tumour_like_cell_from_normal = subset(adata_all, subset = cell_id == 'p007n:CCTATCGAGATGAACT')

In [47]:
most_DC2_tumour_like_cell_from_normal@meta.data['cell_type_level2'] = 'Tumour'

most_DC2_tumour_like_cell_from_normal@meta.data['cell_type_cell_identity'] = 'Tumour'

most_DC2_tumour_like_cell_from_normal@meta.data['cell_identity_resolved_tumour'] = 'Tumour'

In [ ]:
most_DC2_tumour_like_cell_from_normal@meta.data

In [ ]:
adata_GT_TME

In [ ]:
adata_GN_noTCs

In [ ]:
adata_n_noTCs

In [52]:
adata_n = merge(adata_n_noTCs,most_DC2_tumour_like_cell_from_normal)

In [53]:
adata_t = merge(adata_GT_TME, adata_GN_noTCs)

#### make cellchat obj

In [ ]:
cellchat_tumour = createCellChat(object = adata_t, group.by = "cell_identity_resolved_tumour")

In [ ]:
cellchat_normal = createCellChat(object = adata_n, group.by = "cell_identity_resolved_tumour")

In [56]:
cellchat_normal@idents = factor(cellchat_normal@idents,  
                                     levels = cell_type_order_with_Tumour_without_Monocytes)

In [57]:
cellchat_tumour@idents = factor(cellchat_tumour@idents,  
                                     levels = cell_type_order_with_Tumour_without_Monocytes)

In [58]:
CellChatDB <- CellChatDB.human

In [60]:
# use all pathways 
CellChatDB.use <- CellChatDB
cellchat_normal@DB <- CellChatDB.use
cellchat_tumour@DB <- CellChatDB.use

In [61]:
# filter to only genes exist in the database
cellchat_normal <- subsetData(cellchat_normal) 
cellchat_tumour <- subsetData(cellchat_tumour)

In [62]:
plan("multisession", workers = 8)
# Error: Strategy 'multiprocess' is defunct in future (>= 1.32.0) [2023-03-06]. 
# Instead, explicitly specify either 'multisession' (recommended) or 'multicore'.

In [63]:
options(future.globals.maxSize = 8000 * 1024^2)

In [64]:
cellchat_normal <- identifyOverExpressedGenes(cellchat_normal)
cellchat_normal <- identifyOverExpressedInteractions(cellchat_normal)

In [65]:
cellchat_tumour <- identifyOverExpressedGenes(cellchat_tumour)
cellchat_tumour <- identifyOverExpressedInteractions(cellchat_tumour)

In [ ]:
cellchat_normal <- computeCommunProb(cellchat_normal)

In [ ]:
cellchat_tumour <- computeCommunProb(cellchat_tumour)

In [ ]:
# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
cellchat_normal <- filterCommunication(cellchat_normal, min.cells = 10)
cellchat_tumour <- filterCommunication(cellchat_tumour, min.cells = 10)

In [88]:
cellchat_normal <- computeCommunProbPathway(cellchat_normal)
cellchat_tumour <- computeCommunProbPathway(cellchat_tumour)

In [89]:
cellchat_normal <- aggregateNet(cellchat_normal)
cellchat_tumour <- aggregateNet(cellchat_tumour)

In [90]:
groupSize_normal <- as.numeric(table(cellchat_normal@idents))
groupSize_tumour <- as.numeric(table(cellchat_tumour@idents))

In [91]:
cellchat_normal <- netAnalysis_computeCentrality(cellchat_normal, slot.name = "netP")
cellchat_tumour <- netAnalysis_computeCentrality(cellchat_tumour, slot.name = "netP") 
# the slot 'netP' means the inferred intercellular communication network of signaling pathways

#### plot

In [ ]:
options(repr.plot.width=10, 
        repr.plot.height=10,
        repr.plot.res = 200)

netVisual_circle(cellchat_tumour@net$count, vertex.weight = groupSize_tumour, 
                 weight.scale = T, label.edge= F, title.name = "Number of interactions", 
                 color.use = cell_type_order_with_Tumour_without_Monocytes_color)

In [ ]:
netVisual_circle(cellchat_normal@net$count, vertex.weight = groupSize_normal, 
                 weight.scale = T, label.edge= F, title.name = "Number of interactions", 
                 color.use = cell_type_order_with_Tumour_without_Monocytes_color)

In [ ]:
mat <- cellchat_tumour@net$weight
par(mfrow = c(2,2), xpd=TRUE)
for (i in 1:nrow(mat)) {
  mat2 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat2[i, ] <- mat[i, ]
  netVisual_circle(mat2, vertex.weight = groupSize_tumour, weight.scale = T, edge.weight.max = max(mat), 
                   title.name = rownames(mat)[i],
                   color.use = cell_type_order_with_Tumour_without_Monocytes_color)
}

In [ ]:
mat <- cellchat_normal@net$weight
par(mfrow = c(2,2), xpd=TRUE)
for (i in 1:nrow(mat)) {
  mat2 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat2[i, ] <- mat[i, ]
  netVisual_circle(mat2, vertex.weight = groupSize_normal, weight.scale = T, edge.weight.max = max(mat), 
                   title.name = rownames(mat)[i],
                color.use = cell_type_order_with_Tumour_without_Monocytes_color
) 
}

### pathway

This hierarchical plot consist of two components: 
the left portion shows autocrine and paracrine signaling to certain cell groups of interest (i.e, the defined vertex.receiver), 
and the right portion shows autocrine and paracrine signaling to the remaining cell groups in the dataset.

####  pathways that have significant communication

In [ ]:
sort(cellchat_normal@netP$pathways) # 92

In [ ]:
sort(cellchat_tumour@netP$pathways) # 100

In [98]:
shared_sig_pathways = intersect(sort(cellchat_normal@netP$pathways), sort(cellchat_tumour@netP$pathways))

In [ ]:
shared_sig_pathways

In [100]:
pathways.show <- 'BMP'

In [ ]:
length(levels(cellchat_tumour@idents))

In [ ]:
data.frame(levels(cellchat_tumour@idents), c(1:20)) #%>% 
#filter(str_detect(levels.cellchat_tumour.idents., 'genomically_normal'))

In [104]:
# cell group index: all epi in NT
#NT_epi = c(6,7,9,10,17,18,20)
NT_epi = c(13,14,15,16,17,18,19,20)

In [ ]:
netVisual_aggregate(cellchat_tumour, signaling = pathways.show, 
                    vertex.receiver = NT_epi, layout = 'hierarchy')

In [ ]:
options(repr.plot.width=7, 
        repr.plot.height=7,
        repr.plot.res = 200)

netVisual_heatmap(cellchat_tumour, color.heatmap = "Reds", measure = 'weight')

In [ ]:
options(repr.plot.width=7, 
        repr.plot.height=7,
        repr.plot.res = 200)

netVisual_heatmap(cellchat_normal, color.heatmap = "Reds")

In [ ]:
netVisual_heatmap(cellchat_normal, color.heatmap = "Reds", measure = 'weight')

In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_heatmap(cellchat_tumour, signaling = pathways.show, color.heatmap = "Reds") 

In [ ]:
options(repr.plot.width=4, 
        repr.plot.height=4,
        repr.plot.res = 200)

netAnalysis_contribution(cellchat_tumour, signaling = pathways.show)

In [ ]:
options(repr.plot.width=14, 
        repr.plot.height=5,
        repr.plot.res = 300)

netAnalysis_signalingRole_network(cellchat_tumour, signaling = pathways.show, 
                                  width = 25, height = 5, 
                                  font.size = 10) 

In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

netAnalysis_signalingRole_scatter(cellchat_tumour)

In [ ]:
options(repr.plot.width=6, 
        repr.plot.height=5,
        repr.plot.res = 200)

netAnalysis_signalingRole_scatter(cellchat_tumour, signaling = c("NOTCH"))

In [ ]:
options(repr.plot.width=14, 
        repr.plot.height=5,
        repr.plot.res = 300)

netAnalysis_signalingRole_network(cellchat_tumour, signaling = 'NOTCH', 
                                  width = 25, height = 5, 
                                  font.size = 10) 

In [ ]:
options(repr.plot.width=5, 
        repr.plot.height=6,
        repr.plot.res = 200)

netAnalysis_contribution(cellchat_tumour, signaling = 'NOTCH')

In [ ]:
options(repr.plot.width=14, 
        repr.plot.height=20,
        repr.plot.res = 200)

netAnalysis_signalingRole_heatmap(cellchat_tumour, pattern = "outgoing", 
                                  width = 30, height = 40, font.size = 10#,
                                  #signaling = shared_sig_pathways
                                 )

In [ ]:
options(repr.plot.width=14, 
        repr.plot.height=20,
        repr.plot.res = 200)

netAnalysis_signalingRole_heatmap(cellchat_tumour, pattern = "incoming", 
                                  width = 30, height = 40, font.size = 10#,
                                  #signaling = shared_sig_pathways
                                 )

### merge objs

In [ ]:
# merge
object.list <- list(Normal = cellchat_normal, 
                    Tumour = cellchat_tumour
                   )

cellchat <- mergeCellChat(object.list, add.names = names(object.list), cell.prefix = FALSE)


In [ ]:
cellchat 

In [121]:
cellchat@idents$joint = factor(cellchat@idents$joint, 
                               levels = cell_type_order_with_Tumour_without_Monocytes)
cellchat@idents$Normal = factor(cellchat@idents$Normal, 
                               levels = cell_type_order_with_Tumour_without_Monocytes)
cellchat@idents$Tumour = factor(cellchat@idents$Tumour, 
                               levels = cell_type_order_with_Tumour_without_Monocytes)


In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

gg1 <- compareInteractions(cellchat, show.legend = F, group = c(1,2), color.use = c('#808080','#ff7f0e'),
                           digits = 2, size.text = 18)

gg2 <- compareInteractions(cellchat, show.legend = F, group = c(1,2), measure = "weight",
                           color.use = c('#808080','#ff7f0e'),
                           digits = 2, size.text = 18)
gg1 + gg2

In [ ]:
colnames(cellchat@net[['Tumour']][['count']])

In [ ]:
colnames(cellchat@net[['Normal']][['count']])

In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_heatmap(cellchat, width = 16, font.size.title = 15, font.size = 12,
                  color.use = cell_type_order_with_Tumour_without_Monocytes_color
)

In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_heatmap(cellchat, measure = "weight", width = 16, font.size.title = 15, font.size = 12,
                 color.use = cell_type_order_with_Tumour_without_Monocytes_color)

In [ ]:
options(repr.plot.width=16, 
        repr.plot.height=8,
        repr.plot.res = 200)

weight.max <- getMaxWeight(object.list, attribute = c("idents","count"))

par(mfrow = c(1,2), xpd=TRUE)

for (i in 1:length(object.list)) {
  netVisual_circle(object.list[[i]]@net$count, weight.scale = T, label.edge= F, 
                   edge.weight.max = weight.max[2], edge.width.max = 12, 
                   #title.name = paste0("Number of interactions - ", names(object.list)[i]),
                  color.use = cell_type_order_with_Tumour_without_Monocytes_color)
}

In [ ]:
# need centr
options(repr.plot.width=16, 
        repr.plot.height=8,
        repr.plot.res = 200)

num.link <- sapply(object.list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})

#weight.MinMax <- c(min(num.link), max(num.link)) # control the dot size in the different datasets

gg <- list()
for (i in 1:length(object.list)) {
  gg[[i]] <- netAnalysis_signalingRole_scatter(object.list[[i]], #title = names(object.list)[i], 
                                               font.size.title = 18, font.size = 15, label.size = 5,
                                               color.use = cell_type_order_with_Tumour_without_Monocytes_color
                                               #,weight.MinMax = weight.MinMax
                                              ) + 
    xlim(0, 20) + 
    ylim(0, 12)
}

patchwork::wrap_plots(plots = gg)

#### similarity

Functional similarity: High degree of functional similarity indicates major senders and receivers are similar, and it can be interpreted as the two signaling pathways or two ligand-receptor pairs exhibit similar and/or redundant roles.

In [ ]:
cellchat <- computeNetSimilarityPairwise(cellchat, type = "functional")
#> Compute signaling network similarity for datasets 1 2
cellchat <- netEmbedding(cellchat, type = "functional")
#> Manifold learning of the signaling networks for datasets 1 2
cellchat <- netClustering(cellchat, type = "functional")
#> Classification learning of the signaling networks for datasets 1 2

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_embeddingPairwise(cellchat, type = "functional", label.size = 3)

Structural similarity: A structural similarity was used to compare their signaling network structure, without considering the similarity of senders and receivers. NB: Structural similarity analysis is applicable to multiple datsets with the same cell type composition or the vastly different cell type composition.

In [ ]:
cellchat <- computeNetSimilarityPairwise(cellchat, type = "structural")
#> Compute signaling network similarity for datasets 1 2
cellchat <- netEmbedding(cellchat, type = "structural")
#> Manifold learning of the signaling networks for datasets 1 2
cellchat <- netClustering(cellchat, type = "structural")
#> Classification learning of the signaling networks for datasets 1 2

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_embeddingPairwise(cellchat, type = "structural", label.size = 3.5)

In [ ]:
rankSimilarity(cellchat, type = "functional")

In [ ]:
rankSimilarity(cellchat, type = "structural")

#### information flows (sum of communication probability among all pairs == total weights)
- conserved and context-specifc signalling pathways 

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=16,
        repr.plot.res = 200)

gg1 <- rankNet(cellchat, mode = "comparison", stacked = T, do.stat = TRUE, font.size = 10, measure = "count",
              color.use = c('#808080', '#ff7f0e'))
gg2 <- rankNet(cellchat, mode = "comparison", stacked = F, do.stat = TRUE, font.size = 10, measure = "count",
              color.use = c('#808080', '#ff7f0e'))
gg1 + gg2

In [ ]:
gg3 <- rankNet(cellchat, mode = "comparison", stacked = T, do.stat = TRUE, font.size = 10, measure = "weight",
              color.use = c('#808080', '#ff7f0e'))
gg4 <- rankNet(cellchat, mode = "comparison", stacked = F, do.stat = TRUE, font.size = 10, measure = "weight",
              color.use = c('#808080', '#ff7f0e'))
gg3 + gg4

### pathways vs cell groups

In [138]:
i = 1
pathway.union <- union(object.list[[i]]@netP$pathways, object.list[[i+1]]@netP$pathways)

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=16,
        repr.plot.res = 200)

netAnalysis_signalingRole_heatmap(object.list[[1]], pattern = "all", 
                                        signaling = pathway.union, title = names(object.list)[1],  
                                  width = 20, height = 35, color.heatmap = "OrRd", font.size = 10,
                                 color.use = cell_type_order_with_Tumour_without_Monocytes_color)

In [ ]:
netAnalysis_signalingRole_heatmap(object.list[[2]], pattern = "all", 
                                        signaling = pathway.union, 
                                  title = names(object.list)[2], 
                                  width = 20, height = 35, color.heatmap = "OrRd", font.size = 10,
                                  color.use = cell_type_order_with_Tumour_without_Monocytes_color)

### LR pairs from some cell groups to some cell groups (differences in community prob)

In [ ]:
length(levels(cellchat@idents$joint))

In [142]:
shared_sig_pathways = intersect(sort(cellchat@netP$Normal$pathways), sort(cellchat@netP$Tumour$pathways))

In [ ]:
(shared_sig_pathways) # 88

In [ ]:
data.frame(levels(cellchat@idents$joint), c(1:20))

In [ ]:
# from Tumour to everything
options(repr.plot.width=12, 
        repr.plot.height=25,
        repr.plot.res = 200)

netVisual_bubble(cellchat, sources.use = 20, targets.use = c(1:20),  comparison = c(1, 2), angle.x = 45)

In [ ]:
# from Fibroblast to Stem/Tumour (the targets.use doesnt let me do one cell group...)
options(repr.plot.width=12, 
        repr.plot.height=20,
        repr.plot.res = 200)

netVisual_bubble(cellchat, sources.use = 11, targets.use = c(13,20),  comparison = c(1, 2), angle.x = 45)

### differential at pathway level

In [ ]:
# plot all the significant pathways
options(repr.plot.width=12, 
        repr.plot.height=6,
        repr.plot.res = 200)

pdf(paste0(new_data_folder,'/20231129_cellchat_pathway_cell_groups_signals.pdf'), paper = 'a4r')

for(i in shared_sig_pathways){
    pathways.show = i
    
    weight.max <- getMaxWeight(object.list, slot.name = c("netP"), attribute = pathways.show) 
    # control the edge weights across different datasets
    
    par(mfrow = c(1,2), xpd=TRUE)
    
    for (i in 1:length(object.list)) {
        netVisual_aggregate(object.list[[i]], signaling = pathways.show, layout = "circle", 
                            edge.weight.max = weight.max[1], edge.width.max = 10, 
                            signaling.name = paste(pathways.show, names(object.list)[i]),
                            color.use = cell_type_order_with_Tumour_without_Monocytes_color)
    }
}
dev.off()

### identify dysfunctional signaling by using differential expression

In [ ]:
# define a positive dataset, i.e., the dataset with positive fold change against the other dataset
pos.dataset = "Tumour"
# define a char name used for storing the results of differential expression analysis
features.name = pos.dataset
# perform differential expression analysis
cellchat <- identifyOverExpressedGenes(cellchat, group.dataset = "datasets", 
                                       pos.dataset = pos.dataset, 
                                       features.name = features.name, only.pos = FALSE, 
                                       thresh.pc = 0.1, thresh.fc = 0.1, thresh.p = 1)

#> Use the joint cell labels from the merged CellChat object
# map the results of differential expression analysis onto the inferred cell-cell communications to easily manage/subset the ligand-receptor pairs of interest
net <- netMappingDEG(cellchat, features.name = features.name)
# extract the ligand-receptor pairs with upregulated ligands 
net.up <- subsetCommunication(cellchat, net = net, datasets = "Tumour",
                              ligand.logFC = 0.2, receptor.logFC = NULL)

net.down <- subsetCommunication(cellchat, net = net, datasets = "Normal",
                                ligand.logFC = -0.2, receptor.logFC = NULL)



In [149]:
# keep only expressed (not NA) receptors
net.up = net.up[which(net.up$receptor.logFC != 'NA'),] #1855 

In [150]:
net.down = net.down[which(net.down$receptor.logFC != 'NA'),] #411

In [151]:
gene.up <- extractGeneSubsetFromPair(net.up, cellchat)
gene.down <- extractGeneSubsetFromPair(net.down, cellchat)

In [152]:
pairLR.use.up = net.up[, "interaction_name", drop = F]
pairLR.use.down = net.down[, "interaction_name", drop = F]

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=20,
        repr.plot.res = 200)
#' @param remove.isolate whether remove the entire empty column, i.e., communication between certain cell groups
gg1 <- netVisual_bubble(cellchat, pairLR.use = pairLR.use.up, sources.use = 11, targets.use = 13, 
                        comparison = c(1, 2),  angle.x = 90, remove.isolate = T,
                        title.name = paste0("Up-regulated signaling in ", names(object.list)[2]))

gg2 <- netVisual_bubble(cellchat, pairLR.use = pairLR.use.down, sources.use = 11, targets.use = 13, 
                        comparison = c(1, 2),  angle.x = 90, remove.isolate = T,
                        title.name = paste0("Down-regulated signaling in ", names(object.list)[2]))

gg1 + gg2

In [ ]:
# Chord diagram
options(repr.plot.width=10, 
        repr.plot.height=10,
        repr.plot.res = 200)

netVisual_chord_gene(object.list[[2]], sources.use = 11, targets.use = 13, 
                     slot.name = 'net', net = net.up, lab.cex = 0.8, small.gap = 3.5, 
                     title.name = paste0("Up-regulated signaling in ", names(object.list)[2]))

In [ ]:
netVisual_chord_gene(object.list[[1]], sources.use = 11, targets.use = 13, 
                     slot.name = 'net', net = net.down, lab.cex = 0.8, small.gap = 3.5, 
                     title.name = paste0("Down-regulated signaling in ", names(object.list)[2]))

In [ ]:
## Shut down parallel workers
future::plan("sequential")